In [ ]:

#Always make all imports in the first cell of the notebook, run them all once.
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy import ndimage as ndi
from skimage import data
from skimage import io
from skimage.util import img_as_float   
from skimage.filters import gabor_kernel
from skimage.filters import sobel
from skimage.filters import threshold_otsu
from skimage.util import invert
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import img_as_ubyte
import random
import pandas as pd
from scipy import stats
from heapq import *
from sklearn.neighbors import KNeighborsClassifier
import math
from random import randint
%matplotlib inline


In [ ]:
def IAM_Crop(gray_img, bin_img):
        """
        Detects the bounding box of the handwritten paragraph of the given IAM form image
        and returns a cropped image of it.
        :param gray_img:    the IAM form image to be processed.
        :param bin_img:     binarized IAM form image to be processed.
        :return:            cropped gray and binary images of the handwritten paragraph.
        """

        # Get image dimensions.
        height, width = gray_img.shape

        # Find all contours in the page.
        contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # Minimum contour width to be considered as the black separator line.
        threshold_width =1000
        line_offset = 10

        # Page paragraph boundaries.
        up, down, left, right = 0, height - 1, 0, width - 1

        # Detect the main horizontal black separator lines of the IAM handwriting forms.
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)

            if w < threshold_width:
                continue

            if y < height // 2:
                up = max(up, y + h + line_offset)
            else:
                down = min(down, y - line_offset)

        # Apply erosion to remove noise and dots.
        kernel = np.ones((3, 3), np.uint8)
        eroded_img = cv2.erode(bin_img, kernel, iterations=2)
        gray_img = cv2.erode(gray_img, kernel, iterations=2)

        # Get horizontal and vertical histograms.
        hor_hist = np.sum(eroded_img, axis=1) / 255
        ver_hist = np.sum(eroded_img, axis=0) / 255

        # Detect paragraph white padding.
        while left < right and ver_hist[left] == 0:
            left += 1
        while right > left and ver_hist[right] == 0:
            right -= 1
        while up < down and hor_hist[up] == 0:
            up += 1
        while down > up and hor_hist[down] == 0:
            down -= 2

        up-=50
        left-=50
        right+=50
        # Crop images.
        gray_img = gray_img[up:down + 1, left:right + 1]
        bin_img=bin_img[up:down + 1, left:right + 1]

        # Return the handwritten paragraph
        return gray_img,bin_img

In [ ]:
def horizontal_projections(sobel_image):
    return np.sum(sobel_image, axis=1)  



def find_peak_regions(hpp, divider=2):
    threshold = (np.max(hpp)-np.min(hpp))/divider
    peaks = []
    peaks_index = []
    for i, hppv in enumerate(hpp):
        if hppv < threshold:
            peaks.append([i, hppv])
    return peaks

def get_hpp_walking_regions(peaks_index):
    hpp_clusters = []
    cluster = []
    for index, value in enumerate(peaks_index):
        cluster.append(value)

        if index < len(peaks_index)-1 and peaks_index[index+1] - value > 1:
            hpp_clusters.append(cluster)
            cluster = []

        #get the last cluster
        if index == len(peaks_index)-1:
            hpp_clusters.append(cluster)
            cluster = []
            
    return hpp_clusters

#sobel_image = sobel(img)
##hpp = horizontal_projections(sobel_image)
#plt.plot(hpp)
#plt.show()

In [ ]:
def heuristic(a, b):
    return (b[0] - a[0]) ** 2 + (b[1] - a[1]) ** 2

def astar(array, start, goal):

    neighbors = [(0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]
    close_set = set()
    came_from = {}
    gscore = {start:0}
    fscore = {start:heuristic(start, goal)}
    oheap = []

    heappush(oheap, (fscore[start], start))
    
    while oheap:

        current = heappop(oheap)[1]

        if current == goal:
            data = []
            while current in came_from:
                data.append(current)
                current = came_from[current]
            return data

        close_set.add(current)
        for i, j in neighbors:
            neighbor = current[0] + i, current[1] + j            
            tentative_g_score = gscore[current] + heuristic(current, neighbor)
            if 0 <= neighbor[0] < array.shape[0]:
                if 0 <= neighbor[1] < array.shape[1]:                
                    if array[neighbor[0]][neighbor[1]] == 1:
                        continue
                else:
                    # array bound y walls
                    continue
            else:
                # array bound x walls
                continue
                
            if neighbor in close_set and tentative_g_score >= gscore.get(neighbor, 0):
                continue
                
            if  tentative_g_score < gscore.get(neighbor, 0) or neighbor not in [i[1]for i in oheap]:
                came_from[neighbor] = current
                gscore[neighbor] = tentative_g_score
                fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                heappush(oheap, (fscore[neighbor], neighbor))
                
    return []



In [ ]:
#Scan the paths to see if there are any blockers.

def get_binary(img):
    mean = np.mean(img)
    if mean == 0.0 or mean == 1.0:
        return img

    thresh = threshold_otsu(img)
    binary = img <= thresh
    binary = binary*1
    return binary

def path_exists(window_image):
    #very basic check first then proceed to A* check
    if 0 in horizontal_projections(window_image):
        return True
    
    padded_window = np.zeros((window_image.shape[0],1))
    world_map = np.hstack((padded_window, np.hstack((window_image,padded_window)) ) )
    path = np.array(astar(world_map, (int(world_map.shape[0]/2), 0), (int(world_map.shape[0]/2), world_map.shape[1])))
    if len(path) > 0:
        return True
    
    return False

def get_road_block_regions(nmap):
    road_blocks = []
    needtobreak = False
    
    for col in range(nmap.shape[1]):
        start = col
        end = col+20
        if end > nmap.shape[1]-1:
            end = nmap.shape[1]-1
            needtobreak = True

        if path_exists(nmap[:, start:end]) == False:
            road_blocks.append(col)

        if needtobreak == True:
            break
            
    return road_blocks

def group_the_road_blocks(road_blocks):
    #group the road blocks
    road_blocks_cluster_groups = []
    road_blocks_cluster = []
    size = len(road_blocks)
    for index, value in enumerate(road_blocks):
        road_blocks_cluster.append(value)
        if index < size-1 and (road_blocks[index+1] - road_blocks[index]) > 1:
            road_blocks_cluster_groups.append([road_blocks_cluster[0], road_blocks_cluster[len(road_blocks_cluster)-1]])
            road_blocks_cluster = []

        if index == size-1 and len(road_blocks_cluster) > 0:
            road_blocks_cluster_groups.append([road_blocks_cluster[0], road_blocks_cluster[len(road_blocks_cluster)-1]])
            road_blocks_cluster = []

    return road_blocks_cluster_groups

In [ ]:
def extract_line_from_image(image, lower_line, upper_line):
    lower_boundary = np.min(lower_line[:, 0])
    upper_boundary = np.max(upper_line[:, 0])
    img_copy = np.copy(image)
    r, c = img_copy.shape
    for index in range(c-1):
        img_copy[lower_line[index, 0]:0, index] = 255
        img_copy[r:upper_line[index, 0], index] = 255
    
    return img_copy[lower_boundary:upper_boundary, :]

In [ ]:
def Segment(imgpath):
   
    path = imgpath

    imgStar = rgb2gray(imread(path))
    imgStar = img_as_ubyte(imgStar)
    imgStar=cv2.resize(imgStar, (0,0), fx=0.6, fy=0.6) 
    l_padding = 10
    r_padding = 10 
    gray_img = imgStar[:, l_padding:-r_padding]
    thresh, bin_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    img,bin_img = IAM_Crop(gray_img,bin_img)

    img = np.invert(bin_img)



    plt.imshow(img,cmap='gray')
    plt.axis("off")
    plt.title("Gray Image After PreProcessing")
    plt.show()
    # img = gray_img
    
    sobel_image = sobel(img)
    hpp = horizontal_projections(sobel_image)
    #print (np.max(hpp))

    peaks = find_peak_regions(hpp)

    peaks_index = np.array(peaks)[:,0].astype(int)
    count=0
    segmented_img = np.copy(img)
    r,c = segmented_img.shape
    for ri in range(r):
        if ri in peaks_index:
            segmented_img[ri, :] = 0
        
    hpp_clusters = get_hpp_walking_regions(peaks_index)
    binary_image = get_binary(img)

    for cluster_of_interest in hpp_clusters:
        nmap = binary_image[cluster_of_interest[0]:cluster_of_interest[len(cluster_of_interest)-1],:]
        if len(cluster_of_interest)==1:
            continue
        road_blocks = get_road_block_regions(nmap)
        road_blocks_cluster_groups = group_the_road_blocks(road_blocks)
        #create the doorways
        for index, road_blocks in enumerate(road_blocks_cluster_groups):
            window_image = nmap[:, road_blocks[0]: road_blocks[1]+10]
            binary_image[cluster_of_interest[0]:cluster_of_interest[len(cluster_of_interest)-1],:][:, road_blocks[0]: road_blocks[1]+10][int(window_image.shape[0]/2),:] *= 0

    #now that everything is cleaner, its time to segment all the lines using the A* algorithm
    line_segments = []
    for i, cluster_of_interest in enumerate(hpp_clusters):
        nmap = binary_image[cluster_of_interest[0]:cluster_of_interest[len(cluster_of_interest)-1],:]
        path = np.array(astar(nmap, (int(nmap.shape[0]/2), 0), (int(nmap.shape[0]/2),nmap.shape[1]-1)))
        if path.shape == (0,):
            continue
        if i==len(hpp_clusters)-1:
            break
        offset_from_top = cluster_of_interest[0]
        path[:,0] += offset_from_top
        line_segments.append(path)


   
   
      ## add an extra line to the line segments array which represents the last bottom row on the image
    last_bottom_row = np.flip(np.column_stack(((np.ones((img.shape[1],))*img.shape[0]), np.arange(img.shape[1]))).astype(int), axis=0)
    line_segments.append(last_bottom_row)

    line_images=[]
    line_count = len(line_segments)
    #fig, ax = plt.subplots(figsize=(10,10), nrows=line_count-1)
    for line_index in range(line_count-1):
        line_image = extract_line_from_image(img, line_segments[line_index], line_segments[line_index+1])
        line_images.append(line_image)
       # ax[line_index].imshow(line_image, cmap="gray")

    binary=[]    
    for i in range(0,len(line_images)):
        first_line = line_images[i]
        thresh = threshold_otsu(first_line)
        binary.append(first_line > thresh)



    # find the vertical projection by adding up the values of all pixels along rows
    # vertical_projection = np.sum(binary, axis=0)

    # plot the vertical projects
    return binary
    

In [ ]:
def disk_kernel(imgg,loops):
    slopes=[0,0,0]
    arr = np.zeros((loops,2)) #logA(d)-log(d) and log(d)
    for i in range(0,loops):
        d = 2*i+1
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(d,d))        #ndimage.rotate(img, 45) for ellipse
        img_dilation = cv2.erode(np.uint8(imgg), kernel, iterations=1)
        #totalPixels = img_dilation.shape[0]*img_dilation.shape[1]
        Area = cv2.countNonZero(img_dilation)                              ##count white pixels
        arr[i] = (np.log(Area)- np.log(d) , np.log(d))                     #/(Area + cv2.countNonZero(img_dilation))



    min = 1000

    for x in range(1,loops-2):
        for y in range(x+2,loops-1):
            line1 = arr[0:x +1] 
            line2 = arr[x:y +1]
            line3 = arr[y:loops]
        
            slope1,_,_,_,std1 = stats.linregress(line1[:,0], line1[:,1])
            slope2,_,_,_,std2 = stats.linregress(line2[:,0], line2[:,1])
            slope3,_,_,_,std3 = stats.linregress(line3[:,0], line3[:,1])
        
            if(min > std1+std2+std3):
                min = std1+std2+std3
                slopes=[slope1,slope2,slope3]
   # slopes.append(slope1)
    #slopes.append(slope2)
    #slopes.append(slope3)
    return slopes

In [ ]:
def ellipse_kernel(imgg,loops,rotation):
    slopes=[0,0,0]
    arr2= np.zeros((loops,2))
    for i in range(0,loops):
        d = 2*i+1
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2*d,d))
        kernel=ndi.rotate(kernel, rotation) 
        img_dilation = cv2.erode(np.uint8(imgg), kernel, iterations=1)
        Area = cv2.countNonZero(img_dilation)                              ##count white pixels
        arr2[i] = (np.log(Area)- np.log(d) , np.log(d))                     #/(Area + cv2.countNonZero(img_dilation))
        
    min = 1000

    for x in range(1,loops-2):
        for y in range(x+2,loops-1):
            line1 = arr2[0:x +1] 
            line2 = arr2[x:y +1]
            line3 = arr2[y:loops]
        
            slope1,_,_,_,std1 = stats.linregress(line1[:,0], line1[:,1])
            slope2,_,_,_,std2 = stats.linregress(line2[:,0], line2[:,1])
            slope3,_,_,_,std3 = stats.linregress(line3[:,0], line3[:,1])
        
            if(min > std1+std2+std3):
                min = std1+std2+std3
                slopes=[slope1,slope2,slope3]
   # slopes.append(slope1)
    #slopes.append(slope2)
    #slopes.append(slope3)
    return slopes

   

# start of project #

In [ ]:
def Feature_Extraction():
    data = pd.read_csv("data.csv") 
    ######################## randomly choose 7 images for the model and testing###############################################
    classes = random.sample(range(0, 350), 3)
    class1 = data[data["writer-id"] == classes[0]]
    class2 = data[data["writer-id"] == classes[1]]
    class3 = data[data["writer-id"] == classes[2]]

    while(len(class1)<4 or len(class2)<3 or len(class3)<3):
        classes = random.sample(range(0, 350), 3)
        class1 = data[data["writer-id"] == classes[0]]
        class2 = data[data["writer-id"] == classes[1]]
        class3 = data[data["writer-id"] == classes[2]]

    one = np.array(class1)
    two = np.array(class2)
    three = np.array(class3)

    images1 = random.sample(range(0, len(class1)-1), 3)
    images2 = random.sample(range(0, len(class2)-1), 2)
    images3 = random.sample(range(0, len(class3)-1), 2)
    
    image01=Segment('C:/Users/Lenovo/Desktop/formsA-D/'+one[0][0]+'.png')   #class 1
    print('1-done')
    image02=Segment('C:/Users/Lenovo/Desktop/formsA-D/'+one[1][0]+'.png')
    print('2-done')
    image11=Segment('C:/Users/Lenovo/Desktop/formsA-D/'+two[0][0]+'.png')
    print('3-done')
    image12=Segment('C:/Users/Lenovo/Desktop/formsA-D/'+two[1][0]+'.png')   #class 2
    print('4-done')

    image21=Segment('C:/Users/Lenovo/Desktop/formsA-D/'+three[0][0]+'.png') #class 3
    print('5-done')
    image22=Segment('C:/Users/Lenovo/Desktop/formsA-D/'+three[1][0]+'.png')
    print('6-done')
    
    images = []
    images.append(image01)
    images.append(image02)
    images.append(image11)
    images.append(image12)
    images.append(image21)
    images.append(image22)
    
    Y_List = []
    Y_classes = [classes[0],classes[0],classes[1],classes[1],classes[2],classes[2]]
    Y_test = classes[0]
    test_image = one[2][0]
    
    for i in range(0,len(images)):
        for x in range(0, len(images[i])):
            Y_List.append(Y_classes[i])

    Y_train = np.array(Y_List)
    ####################################
    gradientsList = []
    for j in range(0, len(images)):
        for i in range(0, len(images[j])):
            slopes = disk_kernel(images[j][i],20)
            gradients = []
            gradients.append(slopes[0])
            gradients.append(slopes[1])
            gradients.append(slopes[2])
            rotation=0
            for x in range (0,18):
                slopes = (ellipse_kernel(images[j][i],20,rotation))
                gradients.append(slopes[0])
                gradients.append(slopes[1])
                gradients.append(slopes[2])
                rotation+=10
            gradientsList.append(gradients)
        print('1-fetures done')
    X_train = np.array(gradientsList)
    
    return X_train, Y_train, test_image, Y_test
    ####t=np.count_nonzero(np.isnan(X_train))

# print("--------------------")
# print(one[0][0])
# print(one[1][0])
# print(one[2][0])
# print(two[0][0])
# print(two[1][0])
# print(three[0][0])
# print(three[1][0])

In [ ]:
# image01=Segment('G:/imgs/'+one[0][0]+'.png')   #class 1
# image02=Segment('G:/imgs/'+one[1][0]+'.png')

# image11=Segment('G:/imgs/'+two[0][0]+'.png')
# image12=Segment('G:/imgs/'+two[1][0]+'.png')   #class 2

# image21=Segment('G:/imgs/'+three[0][0]+'.png') #class 3
# image22=Segment('G:/imgs/'+three[1][0]+'.png')

In [ ]:
# images = []
# images.append(image01)
# images.append(image02)
# images.append(image11)
# images.append(image12)
# images.append(image21)
# images.append(image22)

In [ ]:
# Y_List = []
# Y_classes = [classes[0],classes[0],classes[1],classes[1],classes[2],classes[2]]
# Y_actual = classes[0]
# for i in range(0,len(images)):
#     for x in range(0, len(images[i])):
#         Y_List.append(Y_classes[i])

# Y_train = np.array(Y_List)

In [ ]:
#print(Y_train)

In [ ]:
# gradientsList = []
# for j in range(0, len(images)):
#     for i in range(0, len(images[j])):
#         slopes = disk_kernel(images[j][i],20)
#         gradients = []
#         gradients.append(slopes[0])
#         gradients.append(slopes[1])
#         gradients.append(slopes[2])
#         rotation=0
#         for x in range (0,18):
#             slopes = (ellipse_kernel(images[j][i],20,rotation))
#             gradients.append(slopes[0])
#             gradients.append(slopes[1])
#             gradients.append(slopes[2])
#             rotation+=10
#         gradientsList.append(gradients)
        
#print(np.array(gradientsList).shape)

In [ ]:
# X_train = np.array(gradientsList)
# print(X_train)
# t=np.count_nonzero(np.isnan(X_train))
# print(t)

In [ ]:
# print(Y_train.shape)
# print(X_train.shape)

In [ ]:
def testing(X_train, Y_train, X_test):
    KNN = KNeighborsClassifier(n_neighbors=5)
    KNN.fit(X_train, Y_train)

    imagesTest = []
    imagesTest.append(Segment('C:/Users/Lenovo/Desktop/formsA-D/'+X_test+'.png')) #test sample

    gradientsList = []
    for j in range(0, len(imagesTest)):
        for i in range(0, len(imagesTest[j])):
            slopes = disk_kernel(imagesTest[j][i],20)
            gradients = []
            gradients.append(slopes[0])
            gradients.append(slopes[1])
            gradients.append(slopes[2])
            rotation=0
            for x in range (0,18):
                slopes = (ellipse_kernel(imagesTest[j][i],20,rotation))
                gradients.append(slopes[0])
                gradients.append(slopes[1])
                gradients.append(slopes[2])
                rotation+=10
            gradientsList.append(gradients)

    X_test = np.array(gradientsList)
    ###########################################

    Y_predict = KNN.predict(X_test)
    #result = np.bincount(Y_predict)
    return Y_predict

In [ ]:
# accuracy = 0

# # try:
  
# X_train, Y_train, test_image, Y_actual = Feature_Extraction()
# result = testing(X_train, Y_train, test_image)
# maximum=np.bincount(result).argmax()

# # except:
# #     maximum=randint(1, 3)

# print(Y_actual)
# print(maximum)



In [ ]:

accuracy = 0
for i in range(0,10):
    print(i)
    try:
    
        X_train, Y_train, test_image, Y_actual = Feature_Extraction()
        result = testing(X_train, Y_train, test_image)
        maximum=np.bincount(result).argmax()

    except:
        maximum=randint(1, 3)

    print(Y_actual)
    print(maximum)
    if(Y_actual==maximum):
        accuracy+=1

In [ ]:
print((accuracy/9)*100)